# Práctica 1: Web Crawling
Guillermo Segura Gómez

Ingresar a la página del gobierno de México, concretamente la versión estenográfica de la mañanera del presidente. Hacer un scraper para obtener información de todas las conferencias. Se tiene primero que extraer la información de las url, luego de cada url extraer la data de lo que se dice. Obtener los datos mas limpios que se pueda. 


### Scraping vs Crawling

* Web Crawling: Es el proceso de navegar por internet y recolectar información sobre las páginas web. Un crawler, también conocido como spider o bot, visita automáticamente las páginas web y sigue los enlaces (hipervínculos) que encuentra en estas páginas. El objetivo principal de un crawler es indexar la información de las páginas web para que puedan ser recuperadas más tarde por un motor de búsqueda. Googlebot es un ejemplo de un crawler muy conocido.

* Web Scraping: Es el proceso de extraer datos específicos de páginas web. A diferencia del crawling, que puede ser más general en la recolección de información, el scraping está orientado a obtener detalles específicos como precios de productos, información de contacto, textos, etc. Para hacer scraping, usualmente se necesita analizar el HTML de la página y extraer los datos necesarios.

Vamos a realizar un crawling para navegar la información y un scraping para obtener el texto. 

## Ligas de cada conferencia

Cargamos la liga general de la cual obtendremos todas las ligas. Utilizamos headers para que se nos reconozca como una computadora y ejecutar el crawling.

In [1]:
# Libreria para hacer solicitudes http
import requests

In [2]:
url_general = 'https://presidente.gob.mx/secciones/version-estenografica/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'
}
response_general = requests.get(url_general, headers=headers)

El objeto que obtenemos es un tipo **response**. 

### ¿Que es un objeto tipo response?

**Instancia de la Clase `Response`**: Cuando se hace una solicitud como `requests.get(url)`, la función devuelve un objeto de la clase `Response`. Este objeto contiene múltiples datos y métodos útiles para trabajar con la respuesta del servidor.

### Contenido del Objeto `Response`

El objeto `Response` incluye varios atributos y métodos útiles, como:

- **`.text`**: El contenido de la respuesta, en forma de cadena de texto (string). Si la respuesta es HTML, por ejemplo, aquí obtendrás el código HTML completo de la página.

- **`.content`**: El contenido de la respuesta en formato binario (bytes). Esto es útil para datos no textuales como imágenes.

- **`.json()`**: Si la respuesta es JSON, puedes usar este método para analizarla automáticamente y convertirla en un diccionario de Python. Por ejemplo, si accedes a un endpoint de una API que devuelve JSON, harías `response.json()` para obtener los datos en un formato manejable.

- **`.status_code`**: El código de estado HTTP de la respuesta (como 200 para éxito, 404 para no encontrado, etc.).

- **`.headers`**: Un diccionario que contiene los encabezados HTTP de la respuesta.

In [3]:
# Se guarda el contenido de la página en un archivo.txt
with open('Conferencias_General.txt', 'w', encoding=response_general.encoding) as file:
    file.write(response_general.text)

Para limpiar los datos se puede utilizar una **expresión regular** que consiste en un string con el que se van a comparar los elementos de la página web. 
Ej de expresión regular `(http[s]?://[^"]+)"`

Otra opción es utilizar la librería BeautifulSoup, muy utilizada para limpiar datos de páginas web. 
**Beautiful Soup**: Beautiful Soup es una biblioteca de Python para extraer datos de documentos HTML y XML. Esta biblioteca crea un árbol con todos los elementos del documento y puede ser utilizado para extraer información. Por lo tanto, esta biblioteca es útil para realizar web scraping.

In [4]:
from bs4 import BeautifulSoup

In [5]:
data_general = response_general.text
soup = BeautifulSoup(data_general, 'html.parser')

La librería genera un objeto tipo `BeautifulSoup`, similar a como se genera un objeto tipo `response`. Sobre el objeto `BeautifulSoup` al igual que con el `response` podemos hacer operaciones. 

Nos interesan solo las url que tienen la estructura de la versión estenográfica o contienen esta palabra clave dentro de la url. `https://presidente.gob.mx/22-01-24-version-estenografica.../`

Para encontrar todos los url necesitamos iterar sobre cada elemento del objeto soup (que contiene las url). Luego comparamos para que contengan las palabras clave. Para hacer esto podemos hacer uso de una list comprenhension.

In [6]:
# Encontrar todos los enlaces
urls_1 = [link.get('href') for link in soup.find_all('a') if link.get('href') 
          and 'version-estenografica-de-la' in link.get('href')]

### Estructura Básica de una Comprensión de Lista

Una comprensión de lista en Python tiene la siguiente forma básica:

```python
[expresión for elemento in iterable if condición]
```

- **`expresión`**: Es lo que cada elemento de la lista resultante será. Puede ser el mismo elemento del iterable o alguna transformación del mismo.
- **`for elemento in iterable`**: Es un bucle `for` que recorre cada elemento en un iterable (como una lista, un rango, etc.).
- **`if condición`**: Es una condición opcional. Si se incluye, solo los elementos del iterable que cumplan con esta condición serán considerados.

### Ejemplo específico

- **`for link in soup.find_all('a')`**: Itera sobre cada elemento `link` (etiqueta `<a>`) en el objeto `soup`.

- **`link.get('href')`**: Obtiene el valor del atributo `href` de cada etiqueta `<a>`.

- **`if link.get('href')`**: Verifica si la etiqueta `<a>` tiene un atributo `href` no nulo.

- **`'version-estenografica-de-la' in link.get('href')`**: Verifica si el texto `'version-estenografica-de-la'` está presente en el valor de `href`.

Ahora necesitamos hacer esto para todas las páginas. Para esto utilizamos un ciclo for. Además agregamos una barra de progreso.

In [17]:
from tqdm import tqdm
import time

# Iteramos sobre todas las páginas
NPaginas = 135

url_general = 'https://presidente.gob.mx/secciones/version-estenografica/page/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'
}

urls_conferencia = [] # Inicializamos la lista

for i in tqdm(range(NPaginas), desc="Procesando páginas"):
    # Obtener cada pagina
    response_i = requests.get(url_general + str(i+1) + '/', headers=headers)
    
    # Filtrar paginas web
    data_i = response_i.text
    soup_i = BeautifulSoup(data_i, 'html.parser')

    # Guardar urls en lista
    urls_conferencia += [link.get('href') for link in soup_i.find_all('a') if link.get('href') 
          and 'version-estenografica-de-la' in link.get('href')]
    
    time.sleep(1)  # Pausa de 1 segundo para no sobrecargar el servidor

Procesando páginas: 100%|██████████| 135/135 [05:13<00:00,  2.32s/it]


In [18]:
# Guardado de url
nombre_archivo = 'urls_conferencia.txt'

# Abrir el archivo en modo de escritura ('w') y guardar cada URL
with open(nombre_archivo, 'w', encoding='utf-8') as archivo:
    for url in urls_conferencia:
        archivo.write(url + '\n')

print(f'Se han guardado {len(urls_conferencia)} URLs en {nombre_archivo}')

Se han guardado 2440 URLs en urls_conferencia.txt


## Información de las páginas

En esta sección necesitamos obtener información de las páginas. Vamos a filtar el html de cada página utilizando la librería beautifulSoup.

Es necesario guardar toda las las páginas contenidas en el arreglo `urls_conferencia`. Previo a esto, se realiza la limpieza para una sola de las páginas. 

In [10]:
# Scraper al primer elemento de la lista
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'
}
response_conferencia = requests.get(urls_conferencia[2400], headers=headers)

In [11]:
import re

# Se edita el contenido html de
data = response_conferencia.text
soup = BeautifulSoup(data, 'html.parser')

# Extraer el texto
texto = soup.get_text()

# Reemplazar dos o más saltos de línea consecutivos con un solo salto de línea
texto_limpio = re.sub(r'\n{2,}', '\n', texto)

# Se guarda el contenido de la página en un archivo.txt
with open('Conferencia_1.txt', 'w', encoding='utf-8') as file:
    file.write(texto_limpio)

Analizando el archivo que se obtiene (y otros mas) nos damos cuenta de que la conferencia comienza siempre con el presidente al habla y termina después de tres guiones (---). Podemos filtrar aún mas los datos para obtener únicamente el contenido de la rueda de prensa. El comienzo lo filtramos con la etiqueta `Search`.

Para el final de cada archivo es mas complicado, en la mayoria tenemos los tres guiones, pero en los archivos el inicio utilizan otra nomenclatura. Vamos a tratar de incluir la mayoría de nomenclaturas.

In [12]:
# Dividir el texto en el punto donde aparece search (y otros elementos) y tomamos lo siguiente
texto_limpio = texto_limpio.split('Search')[1]
texto_limpio = texto_limpio.split('---')[0]
texto_limpio = texto_limpio.split('+++++')[0]
texto_limpio = texto_limpio.split('– – – 0 – – –')[0]

with open('Conferencia_1.txt', 'w', encoding='utf-8') as file:
    file.write(texto_limpio)

Ya que editamos y sabemos como se van a guardar los archivos, hacemos el ciclo completo.

In [23]:
import os
from tqdm import tqdm

# La carpeta 'MorningData' existe
if not os.path.exists('MorningData'):
    os.makedirs('MorningData')

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'
}

# Función para leer URLs desde un archivo
def leer_urls(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        return f.read().splitlines()

# Función para guardar la lista actualizada de URLs
def guardar_urls(archivo, urls):
    with open(archivo, 'w', encoding='utf-8') as f:
        for url in urls:
            f.write(url + '\n')

try:

    # Leer las URLs desde el archivo
    urls_conferencia = leer_urls('urls_conferencia.txt')
    urls_procesadas = []

    for i, url in tqdm(enumerate(urls_conferencia), desc="Procesando páginas"):
        # Hacer la request
        response_i = requests.get(url, headers=headers)
        data_i = response_i.text

        # Extraer y limpiar el texto
        soup_i = BeautifulSoup(data_i, 'html.parser')
        texto_i = soup_i.get_text()
        texto_limpio_i = re.sub(r'\n{2,}', '\n', texto_i)

        texto_limpio_i = texto_limpio_i.split('Search')[1]
        texto_limpio_i = texto_limpio_i.split('---')[0]
        texto_limpio_i = texto_limpio_i.split('+++++')[0]
        texto_limpio_i = texto_limpio_i.split('– – – 0 – – –')[0]

        # Guardar los elementos
        title_i = texto_limpio_i[1:9]

        # Guardar en un archivo de texto
        with open(f'MorningData/' + title_i + '.txt', 'w', encoding='utf-8') as file:
            file.write(texto_limpio_i)
        
        # Marcar la URL como procesada
        urls_procesadas.append(url)

finally:
    # Eliminar URLs procesadas de la lista original y guardar la lista actualizada
    urls_restantes = [url for url in urls_conferencia if url not in urls_procesadas]
    guardar_urls('urls_conferencia.txt', urls_restantes)


Procesando páginas: 2142it [36:30,  1.02s/it]
